In [1]:
import pandas as pd
import numpy as np
import os 
import pandas as pds
from dask import dataframe
import re
import numpy as np
import seaborn as sbn

from matplotlib import pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer

In [306]:
train = pd.read_csv("train.csv")
validation = pd.read_csv("validation_sample.csv")
test= pd.read_csv("test.csv")

In [3]:
train.head(39)

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...
5,5,1,type=SYSCALL msg=audit(1611889244.855:247124):...
6,6,0,"Jan 22 06:28:59 localhost kibana: {""type"":""err..."
7,7,0,Jan 19 09:19:14 localhost logstash: [2021-01-1...
8,8,0,"Jan 20 08:16:43 localhost kibana: {""type"":""err..."
9,9,0,"Jan 19 09:26:15 localhost kibana: {""type"":""log..."


## LEVEL 이 나타나는 빈도
+ 0 이 제일 적은 것을 알 수 있음
+ 2,4,6 짝수 등급은 매우 낮은 빈도를 보여줌

In [6]:
train.head()

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...


In [25]:
train[train['level'] == 2].head(30)

,id,level,full_log
39057,39057,2,The average number of logs between 10:00 and 1...
87600,87600,2,The average number of logs between 5:00 and 6:...
109510,109510,2,The average number of logs between 6:00 and 7:...
262892,262892,2,The average number of logs between 21:00 and 2...
294072,294072,2,The average number of logs between 5:00 and 6:...
294127,294127,2,The average number of logs between 4:00 and 5:...
343011,343011,2,The average number of logs between 14:00 and 1...
345903,345903,2,The average number of logs between 15:00 and 1...
365348,365348,2,Sep 18 11:02:59 localhost sudo: apache : TTY=...
376681,376681,2,The average number of logs between 2:00 and 3:...


## 보안등급별로 맨앞에 나타나는 단어는 무엇일까?
+ 0번은 월 데이터로 시작하는 경우가 많음
+ 1번은 "type = SYSCALL" 경우가 많고, 다른 보안 레벨에서는 나타나지 않음(정의 가능)
+ 3,5 번에서는 oscap, ossec의 에러가 높은 빈도로 발생

+ 보안등급마다 시작하는 단어가 나눠지는 경우가 꽤 많음
+ 비슷하게 시작하는 단어들의 보안등급을 잘 구별하는게 핵심(날짜, 

In [88]:
train['full_log']=train['full_log'].str.replace(r'[0-9]', '')


In [288]:
#타입 3은ans = log[4:13]
def first(log):
    ans = log[:5]
    return ans

train['word1'] = train['pre_log'].map(first)
test['word1'] = test['pre_log'].map(first)
validation['word1'] = validation['pre_log'].map(first)

In [289]:
train['word1'].value_counts()

Jan l    207025
type     116868
Feb l     35544
Dec l     25882
Oct l     23883
Sep l     23193
Nov l     21633
Mar l     12174
level      2331
ossec      1318
File        822
Syste       820
error       459
oscap       366
E pro       297
Nov s       238
NTFS         25
 Feb         19
Troja        15
Windo        13
 MARK        12
The a        11
 Oct          9
junip         5
Nov d         4
 Jan          3
OpenS         3
Name: word1, dtype: int64

# 타입별로 모아보기 ( 7번 분류용)

In [290]:
word = 'oscap'
train_type1 = train[train['word1'] == word]
validation_type1 = validation[validation['word1'] == word]
test_type1 = test[test['word1'] == word]

In [291]:
train_type1

,id,level,full_log,pre_log,first,word1
3396,3396,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016030...",oscap msg xccdf result scan id content ssg cen...,oscap msg xccdf result scan id,oscap
3661,3661,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016020...",oscap msg xccdf result scan id content ssg cen...,oscap msg xccdf result scan id,oscap
5658,5658,5,"oscap: msg: ""xccdf-result"", scan-id: ""00016030...",oscap msg xccdf result scan id content ssg cen...,oscap msg xccdf result scan id,oscap
5695,5695,5,"oscap: msg: ""xccdf-result"", scan-id: ""00016019...",oscap msg xccdf result scan id content ssg cen...,oscap msg xccdf result scan id,oscap
7206,7206,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016052...",oscap msg xccdf result scan id content ssg cen...,oscap msg xccdf result scan id,oscap
...,...,...,...,...,...,...
466619,466619,5,"oscap: msg: ""xccdf-result"", scan-id: ""00016007...",oscap msg xccdf result scan id content ssg cen...,oscap msg xccdf result scan id,oscap
467561,467561,5,"oscap: msg: ""xccdf-result"", scan-id: ""00016019...",oscap msg xccdf result scan id content ssg cen...,oscap msg xccdf result scan id,oscap
468063,468063,3,"oscap: msg: ""xccdf-result"", scan-id: ""00016020...",oscap msg xccdf result scan id content ssg cen...,oscap msg xccdf result scan id,oscap
468233,468233,5,"oscap: msg: ""xccdf-result"", scan-id: ""00016080...",oscap msg xccdf result scan id content ssg cen...,oscap msg xccdf result scan id,oscap


In [292]:
train_type1.reset_index(drop=False,inplace = True)
test_type1.reset_index(drop=False,inplace = True)


In [293]:
del train_type1['first']
del test_type1['first']
del validation_type1['first']

In [294]:
def first(log):
    #ans = log.split(" ")[:30]
    #total_len = len(log)
    #total_len = int(total_len * 0.3)
    ans = log[:100]

    return ans

train_type1['first'] = train_type1['pre_log'].map(first)
test_type1['first'] = test_type1['pre_log'].map(first)
validation_type1['first'] = validation_type1['pre_log'].map(first)

<ipython-input-294-32abc458419c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_type1['first'] = train_type1['pre_log'].map(first)
<ipython-input-294-32abc458419c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_type1['first'] = test_type1['pre_log'].map(first)
<ipython-input-294-32abc458419c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [295]:
del train_type1['full_log']
del test_type1['full_log']
del validation_type1['full_log']

del train_type1['pre_log']
del test_type1['pre_log']
del validation_type1['pre_log']


In [299]:


train_type1.to_csv("train2.csv",index= False)
test_type1.to_csv("test2.csv",index= False)
validation_type1.to_csv("validation2.csv",index= False)


In [296]:
first

<function __main__.first(log)>

In [297]:
test_type1['first'][1]

'oscap msg xccdf result scan id content ssg centos ds xml title Verify boot grub grub cfg User Owners'

In [298]:
train_type1_data = train.iloc[train_type1]
test_type1_Data = test.iloc[test_type1]

ValueError: invalid literal for int() with base 10: 'oscap'

In [19]:
train_type1_data

,id,level,full_log,first,d
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
5,5,1,type=SYSCALL msg=audit(1611889244.855:247124):...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
10,10,1,type=SYSCALL msg=audit(1611895525.367:489785):...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
15,15,1,type=SYSCALL msg=audit(1603072847.573:38657): ...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
22,22,1,type=SYSCALL msg=audit(1603159410.494:164830):...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
...,...,...,...,...,...
472945,472945,1,type=SYSCALL msg=audit(1611893440.465:417650):...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
472951,472951,1,type=SYSCALL msg=audit(1611890997.087:322803):...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
472953,472953,1,type=SYSCALL msg=audit(1611895531.931:497276):...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
472955,472955,1,type=SYSCALL msg=audit(1611887841.572:191604):...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...


In [21]:
train_type1_data['full_log']=train_type1_data['full_log'].str.replace(r'[0-9]', '<num>')
test_type1_Data['full_log']=test_type1_Data['full_log'].str.replace(r'[0-9]', '<num>')

<ipython-input-21-0009adcb6a8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_type1_data['full_log']=train_type1_data['full_log'].str.replace(r'[0-9]', '<num>')
<ipython-input-21-0009adcb6a8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_type1_Data['full_log']=test_type1_Data['full_log'].str.replace(r'[0-9]', '<num>')


In [22]:
train_type1_data

,id,level,full_log,first,d
4,4,1,type=SYSCALL msg=audit(<num><num><num><num><nu...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
5,5,1,type=SYSCALL msg=audit(<num><num><num><num><nu...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
10,10,1,type=SYSCALL msg=audit(<num><num><num><num><nu...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
15,15,1,type=SYSCALL msg=audit(<num><num><num><num><nu...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
22,22,1,type=SYSCALL msg=audit(<num><num><num><num><nu...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
...,...,...,...,...,...
472945,472945,1,type=SYSCALL msg=audit(<num><num><num><num><nu...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
472951,472951,1,type=SYSCALL msg=audit(<num><num><num><num><nu...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
472953,472953,1,type=SYSCALL msg=audit(<num><num><num><num><nu...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...
472955,472955,1,type=SYSCALL msg=audit(<num><num><num><num><nu...,type=SYSCALL,type=SYSCALL msg=audit(.:): arch=ce syscall= s...


In [34]:
train_text=list(train_type1_data['full_log'])
test_text=list(test_type1_Data['full_log'])

In [35]:
#CountVectorizer로 벡터화

#TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=CountVectorizer(analyzer="word", max_features=10000)
#vectorizer=TfidfVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

In [37]:
from sklearn.svm import OneClassSVM
X = train_features
clf = OneClassSVM(gamma='auto').fit(X)
ans = clf.predict(test_features)

In [38]:
test_type1_Data['label'] = pd.Series(ans)

<ipython-input-38-14f73c180abc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_type1_Data['label'] = pd.Series(ans)


In [42]:
test_type1_Data.to_csv("check.csv",index =False)

In [40]:
len(kmeans.labels_)

349290

# 분류 문제 6개가 아닌 2x개로 늘려보기

In [100]:
train

,id,level,full_log
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err..."
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err..."
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err..."
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...
...,...,...,...
472967,472967,0,Feb 28 10:10:06 localhost logstash: 7738 error:
472968,472968,1,type=SYSCALL msg=audit(1611890993.458:321827):...
472969,472969,0,"Oct 12 02:20:29 localhost kibana: {""type"":""log..."
472970,472970,0,"Jan 15 09:02:43 localhost kibana: {""type"":""err..."


In [107]:
train['full_log']=train['full_log'].str.replace(r'[0-9]', '<num>')
test['full_log']=test['full_log'].str.replace(r'[0-9]', '<num>')

In [111]:
train

,id,level,full_log
0,0,0,Sep <num><num> <num><num>:<num><num>:<num><num...
1,1,0,Feb <num> <num><num>:<num><num>:<num><num> lo...
2,2,0,Jan <num><num> <num><num>:<num><num>:<num><num...
3,3,0,Jan <num> <num><num>:<num><num>:<num><num> lo...
4,4,1,type=SYSCALL msg=audit(<num><num><num><num><nu...
...,...,...,...
472967,472967,0,Feb <num><num> <num><num>:<num><num>:<num><num...
472968,472968,1,type=SYSCALL msg=audit(<num><num><num><num><nu...
472969,472969,0,Oct <num><num> <num><num>:<num><num>:<num><num...
472970,472970,0,Jan <num><num> <num><num>:<num><num>:<num><num...


In [110]:
train_text=list(train_type1_data['full_log'])
test_text=list(test_type1_Data['full_log'])

NameError: name 'train_type1_data' is not defined

In [10]:
#CountVectorizer로 벡터화

#TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer=CountVectorizer(analyzer="word", max_features=10000)
#vectorizer=TfidfVectorizer(analyzer="word", max_features=20000)
train_features=vectorizer.fit_transform(train_text)
test_features=vectorizer.transform(test_text)

NameError: name 'train_text' is not defined

# 이상한 문제들 제거

In [153]:
lit = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
def mask(tt):
    tt=tt.apply(lambda x: re.sub(r'(\\n)',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[^a-zA-Zㄱ-ㅣ가-힣0-9:=\s\(\)./,\<\>]+',' ',x))
    tt=tt.apply(lambda x: re.sub(r' ?(?P<note>[:=\(\)./,\<\>]) ?', ' \g<note> ', x))
    tt=tt.apply(lambda x: re.sub(r'[0-9]+',' ',x))
    tt=tt.apply(lambda x: re.sub(r"':/()",' ',x))
    tt=tt.apply(lambda x: re.sub(r':',' ',x))
    tt=tt.apply(lambda x: re.sub(r',',' ',x))
    # = tt.apply(lambda x: re.sub(r'(',' ',x))
    #t = tt.apply(lambda x: re.sub(r')',' ',x))
    tt=tt.apply(lambda x: re.sub(r'[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]',' ',x))
    for st in lit:
        st = " "+st + " "
        tt=tt.apply(lambda x: re.sub(st,' ',x))
    tt=tt.apply(lambda x: re.sub(r'\s+',' ',x))
    
    return tt

In [154]:
validation['pre_log'] = mask(validation.full_log)
train['pre_log'] = mask(train.full_log)
test['pre_log'] = mask(test.full_log)

In [307]:
train = pd.read_csv("mask_train.csv")
test = pd.read_csv("mask_test.csv")
#validation = pd.read_csv("mask_validation.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'mask_validation.csv'

In [310]:
#del train['full_log']
#del test['full_log']
#del validation['full_log']

validation.to_csv("mask_vaild2.csv",index = False)
train.to_csv("mask_train2.csv",index =False)
test.to_csv("mask_test2.csv",index =False)

In [308]:
def first(log):
    #ans = log.split(" ")[:30]
    total_len = len(log)
    total_len = int(total_len * 0.3)
    ans = log[:200]

    return ans

train['first'] = train['pre_log'].map(first)
test['first'] = test['pre_log'].map(first)
#validation['first'] = validation['pre_log'].map(first)

train_count = train['first'].unique()
len(train_count)

test_count = test['first'].unique()
len()

t = train['first'].value_counts()

train3 = train[train['word1'] == 'type=']
valid3 = validation[validation['word1'] == 'type=']


In [177]:
del train['full_log']
del train['pre_log']
del train['word1']

del valid['full_log']
del valid['pre_log']
del valid['word1']

In [179]:
train.to_csv("train4.csv",index =False)
valid.to_csv("valid4.csv",index =False)

In [169]:
validation['pre_log'][0]

'type ANOM PROMISCUOUS msg audit dev enp prom old prom auid uid gid ses type SYSCALL msg audit arch syscall success yes exit aed items ppid pid auid uid gid euid suid fsuid egid sgid fsgid tty none ses comm W enp exe usr sbin suricata subj system system unconfined service key null type PROCTITLE msg audit proctitle F F D F E '

In [87]:
len(train.full_log[1])

379

In [46]:
a = pd.read_csv("mask_vaild.csv")

In [309]:
train

,id,level,full_log,pre_log,first
0,0,0,"Sep 24 10:02:22 localhost kibana: {""type"":""err...",Sep localhost kibana type error timestamp Z ta...,Sep localhost kibana type error timestamp Z ta...
1,1,0,Feb 8 16:21:00 localhost logstash: [2021-02-0...,Feb localhost logstash INFO logstash outputs e...,Feb localhost logstash INFO logstash outputs e...
2,2,0,"Jan 13 01:50:40 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp Z ta...,Jan localhost kibana type error timestamp Z ta...
3,3,0,"Jan 4 10:18:31 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp Z ta...,Jan localhost kibana type error timestamp Z ta...
4,4,1,type=SYSCALL msg=audit(1603094402.016:52981): ...,type SYSCALL msg audit arch syscall success ye...,type SYSCALL msg audit arch syscall success ye...
...,...,...,...,...,...
472967,472967,0,Feb 28 10:10:06 localhost logstash: 7738 error:,Feb localhost logstash error,Feb localhost logstash error
472968,472968,1,type=SYSCALL msg=audit(1611890993.458:321827):...,type SYSCALL msg audit arch syscall success ye...,type SYSCALL msg audit arch syscall success ye...
472969,472969,0,"Oct 12 02:20:29 localhost kibana: {""type"":""log...",Oct localhost kibana type log timestamp Z tags...,Oct localhost kibana type log timestamp Z tags...
472970,472970,0,"Jan 15 09:02:43 localhost kibana: {""type"":""err...",Jan localhost kibana type error timestamp Z ta...,Jan localhost kibana type error timestamp Z ta...
